# Proyecto 2 Analisis Exploratorio

In [1]:
import os
import pandas as pd

In [2]:
data_path = "/kaggle/input/rsna-2022-cervical-spine-fracture-detection"
print(os.listdir(data_path))
train_df = pd.read_csv(f"{data_path}/train.csv")
test_df = pd.read_csv(f"{data_path}/test.csv")
sample_sub = pd.read_csv(f"{data_path}/sample_submission.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
train_df.head() # Aqui tienes un codigo de python para nah no te creas pinche pedro, hoy si borre el comentario

['sample_submission.csv', 'train_images', 'train_bounding_boxes.csv', 'segmentations', 'train.csv', 'test.csv', 'test_images']
Train shape: (2019, 9)
Test shape: (3, 3)


,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0


## Descripción de los Datos y Carga

El dataset de la competencia RSNA 2022 Cervical Spine Fracture Detection contiene imágenes médicas en formato DICOM(.dcm) y archivos de metadatos que permiten relacionar cada estudio con su diagnóstico.  
El objetivo es identificar fracturas en la columna cervical (C1–C7) tanto a nivel vertebral como a nivel paciente.

### Archivos principales

- train.csv  
  Contiene las etiquetas del conjunto de entrenamiento.  
  - StudyInstanceUID: identificador único del estudio (escaneo de un paciente).  
  - patient_overall: variable binaria que indica si el paciente tiene fractura en alguna vértebra (1 = fractura, 0 = sano).  
  - C1 … C7: variables binarias que indican si existe fractura en cada vértebra cervical específica.

- test.csv  
  Define la estructura de predicción para el conjunto de prueba.  
  - row_id: identificador único de fila (se relaciona con sample_submission.csv).  
  - StudyInstanceUID: identificador del estudio en el conjunto de prueba.  
  - prediction_type: cuál de las ocho columnas objetivo (paciente o cada vértebra) requiere predicción.

- sample_submission.csv  
  Archivo de ejemplo para el formato de entrega.  
  - row_id: mismo identificador que en test.csv.  
  - fractured: columna objetivo que debe predecirse (0 = sano, 1 = fractura).

- train_bounding_boxes.csv  
  Contiene coordenadas de bounding boxes que delimitan las vértebras en un subconjunto de los estudios de entrenamiento.

- segmentations/  
  Anotaciones a nivel de píxel (volúmenes NIfTI). Etiquetan cada vértebra cervical (C1–C7) y en algunos casos vértebras torácicas.

- [train/test]_images/[StudyInstanceUID]/[slice_number].dcm  
  Imágenes médicas en formato DICOM (cortes axiales). Cada carpeta corresponde a un estudio, que contiene múltiples cortes.

---

### Carga de archivos en Kaggle Notebook

Utilizamos Kaggle Notebook para correr el proyecto (porque son 300GB en fotos cervicales :eyes: )


## Operaciones de Limpieza

- De los 4 datasets que tenemos los vamos a limpiar pero antes veamos los heads de cada uno

In [3]:
train_df.head()

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0


In [4]:
test_df.head()

,row_id,StudyInstanceUID,prediction_type
0,1.2.826.0.1.3680043.10197_C1,1.2.826.0.1.3680043.10197,C1
1,1.2.826.0.1.3680043.10454_C1,1.2.826.0.1.3680043.10454,C1
2,1.2.826.0.1.3680043.10690_C1,1.2.826.0.1.3680043.10690,C1


In [5]:
df_submission = pd.read_csv(f"{data_path}/sample_submission.csv")

df_submission.head(2000)

,row_id,fractured
0,1.2.826.0.1.3680043.10197_C1,0.5
1,1.2.826.0.1.3680043.10454_C1,0.5
2,1.2.826.0.1.3680043.10690_C1,0.5


In [6]:
df_train_bounding_boxes = pd.read_csv(f"{data_path}/train_bounding_boxes.csv")

df_train_bounding_boxes .head()

,StudyInstanceUID,x,y,width,height,slice_number
0,1.2.826.0.1.3680043.10051,219.27715,216.71419,17.30440,20.38517,133
1,1.2.826.0.1.3680043.10051,221.56460,216.71419,17.87844,25.24362,134
2,1.2.826.0.1.3680043.10051,216.82151,221.62546,27.00959,26.37454,135
3,1.2.826.0.1.3680043.10051,214.49455,215.48637,27.92726,37.51363,136
4,1.2.826.0.1.3680043.10051,214.00000,215.48637,27.00000,43.51363,137


### Analisis de Limpieza

Lo que haremos es un analisis de limpieza de los datos de 2 cosas importantes, si son datos unicos, si faltan datos y su variacion.


In [7]:
import pandas as pd

def data_cleaning_report(df: pd.DataFrame) -> tuple[pd.DataFrame, int]:
    n_rows = len(df)

    # Conteo de valores faltantes y porcentaje
    missing_count = df.isnull().sum()
    missing_pct = (missing_count / n_rows) * 100

    # Tipos de datos
    dtypes = df.dtypes

    # Número de valores únicos por columna
    unique_count = df.nunique()

    # Valores constantes (columnas que son todas iguales)
    constant_cols = [col for col in df.columns if df[col].nunique() == 1]

    # Construir reporte
    report = pd.DataFrame({
        "Tipo": dtypes,
        "Valores_Faltantes": missing_count,
        "Porcentaje_Faltantes": missing_pct.round(2),
        "Valores_Únicos": unique_count,
    })

    report["Es_Constante"] = report.index.isin(constant_cols)

    # Agregar fila resumen
    resumen = pd.DataFrame(
        {"Tipo": "—",
         "Valores_Faltantes": df.isnull().sum().sum(),
         "Porcentaje_Faltantes": (df.isnull().sum().sum() / (n_rows * df.shape[1])) * 100,
         "Valores_Únicos": "—",
         "Es_Constante": "—"},
        index=[f"TOTAL_FILAS={n_rows}"]
    )

    report = pd.concat([report.sort_values(by="Porcentaje_Faltantes", ascending=False), resumen])

    # Retorna el reporte y el total de filas
    return report, n_rows


In [8]:
reporte = data_cleaning_report(train_df)
print("=== Reporte de Calidad de Datos train_df ===")
print(reporte)
reporte_test = data_cleaning_report(test_df)
print("=== Reporte de Calidad de Datos test_df===")
print(reporte_test)

reporte_df_train_bounding_boxes = data_cleaning_report(df_train_bounding_boxes)
print("=== Reporte de Calidad de Datos df_train_bounding_boxes ===")
print(reporte_df_train_bounding_boxes)
reporte_df_submission = data_cleaning_report(df_submission)
print("=== Reporte de Calidad de Datos rdf_submission ===")
print(reporte_df_submission)



=== Reporte de Calidad de Datos train_df ===
(                    Tipo  Valores_Faltantes  Porcentaje_Faltantes  \
StudyInstanceUID  object                  0                   0.0   
patient_overall    int64                  0                   0.0   
C1                 int64                  0                   0.0   
C2                 int64                  0                   0.0   
C3                 int64                  0                   0.0   
C4                 int64                  0                   0.0   
C5                 int64                  0                   0.0   
C6                 int64                  0                   0.0   
C7                 int64                  0                   0.0   
TOTAL_FILAS=2019       —                  0                   0.0   

                 Valores_Únicos Es_Constante  
StudyInstanceUID           2019        False  
patient_overall               2        False  
C1                            2        False  
C2    

Podemos ver que el dataset estan bastante limpios, de hecho no hay ningun datos faltantes. Y de todo podemos ver que el dummision solo tiene 3 filas al igual que test. por lo que los unicos que vamos a limpiar son el df_train_bounding_boxes y el train_df. Y el train realmente no esta sucio, se reviso cada columna y todos estan . Lo que haremos es lo siguiente, convertir cualquier dato de patient_overall y los Cs tenga valores de enteros. 

Con el ultimo veremos lo siguiente para ver si tenemos 



In [9]:
def validate_bounding_boxes(df: pd.DataFrame) -> pd.DataFrame:
    invalid = df[(df["width"] <= 0) | (df["height"] <= 0)]
    print(f"Bounding boxes inválidas: {len(invalid)} de {len(df)}")
    return invalid

# Uso
invalid_boxes = validate_bounding_boxes(df_train_bounding_boxes)


Bounding boxes inválidas: 0 de 7217


Podemos ver que nuestros valores de width y height no son menores o iguales a 0 , por lo que si se pueden usar. Asi que lo que haremos es los floats ponerlos con 5 decimales y los enteros como enteros. 



In [10]:
def normalize_bounding_boxes(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Redondear floats a 5 decimales
    float_cols = ["x", "y", "width", "height"]
    for col in float_cols:
        df[col] = df[col].astype(float).round(5)

    # Asegurar enteros en slice_number
    df["slice_number"] = df["slice_number"].astype(int)

    return df

# --- Normalización train ---
def normalize_train_labels(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Columnas binarias (patient_overall y C1..C7)
    binary_cols = ["patient_overall", "C1", "C2", "C3", "C4", "C5", "C6", "C7"]
    for col in binary_cols:
        df[col] = df[col].astype(int)

    return df

df_boxes_norm = normalize_bounding_boxes(df_train_bounding_boxes)
df_train_norm = normalize_train_labels(train_df)

print(df_boxes_norm.dtypes)
print(df_train_norm.dtypes)

StudyInstanceUID     object
x                   float64
y                   float64
width               float64
height              float64
slice_number          int64
dtype: object
StudyInstanceUID    object
patient_overall      int64
C1                   int64
C2                   int64
C3                   int64
C4                   int64
C5                   int64
C6                   int64
C7                   int64
dtype: object


Como podemos ver ya con esto se han limpiados los datasets de manera correcta. Pero la labor no fue tan dura debido a que ya se llevo a un nivel bastante limpio
